## 피쳐 다 network data에있는거 다넣고 feature engeinnering (rolling 추가) 
피쳐 19개만 고른거랑 시프팅한거보단 못함

In [ ]:
import os
from typing import List, Dict
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
import lightgbm as lgb
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [2]:
# 파일 호출
data_path: str = "data"
train_df: pd.DataFrame = pd.read_csv(os.path.join(data_path, "train.csv")).assign(_type="train") # train 에는 _type = train 
test_df: pd.DataFrame = pd.read_csv(os.path.join(data_path, "test.csv")).assign(_type="test") # test 에는 _type = test
submission_df: pd.DataFrame = pd.read_csv(os.path.join(data_path, "test.csv")) # ID, target 열만 가진 데이터 미리 호출
df: pd.DataFrame = pd.concat([train_df, test_df], axis=0)
df


,ID,target,_type
0,2023-01-01 00:00:00,2.0,train
1,2023-01-01 01:00:00,1.0,train
2,2023-01-01 02:00:00,1.0,train
3,2023-01-01 03:00:00,1.0,train
4,2023-01-01 04:00:00,2.0,train
...,...,...,...
2787,2024-04-26 03:00:00,NaN,test
2788,2024-04-26 04:00:00,NaN,test
2789,2024-04-26 05:00:00,NaN,test
2790,2024-04-26 06:00:00,NaN,test


In [3]:
file_names : List[str]=[
    f for f in os.listdir(data_path) if f.startswith('HOURLY_') and f.endswith(".csv")
]

file_dict: Dict[str,pd.DataFrame]={
    f.replace(".csv",""): pd.read_csv(os.path.join(data_path,f)) for f in file_names
}

for _file_name, _df in tqdm(file_dict.items()):
    _rename_rule={
        col: f"{_file_name.lower()}_{col.lower()}" if col!="datetime" else "ID"
        for col in _df.columns
    }
    _df=_df.rename(_rename_rule,axis=1)
    df=df.merge(_df,on="ID",how="left")


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 107/107 [00:01<00:00, 57.91it/s]


In [4]:
df
missing_values=df.isnull().sum()
missing_values
missing_percentage= (missing_values)/len(df) *100
missing_percentage

sorted_missing_percentage=missing_percentage.sort_values(ascending=False)

# null 100 percent인 feature들
name_null=[]
for col, val in sorted_missing_percentage.items():
    if val==100:
        name_null.append(col)        
len(name_null)
for i in name_null:
    df=df.drop(columns=i)
df.shape


(11552, 215)

In [ ]:
#내일 베이스라인의 선택적인 컬럼만 가져와서 nan값에 넣고 해보기

In [5]:
cols_dict : Dict[str,str] = {}
for i in df.columns:
    
    parts=i.split("_")
    if len(parts)>=3 and 'network-data' in parts:
        cols_dict[i]="_".join(parts[2:])
    elif len(parts) >= 3 and 'market-data' in parts:
        if 'exchange' in parts and 'all' and ('taker-buy-sell-stats' in parts or 'funding-rates' in parts):
            cols_dict[i] = "_".join(parts[2:])
        elif 'all' in parts and 'symbol' in parts and 'exchange' in parts and ('liquidations'  or 'open-interest' in parts):
            cols_dict[i] = "_".join(parts[2:])
        elif 'coinbase-premium-index' in parts:
            cols_dict[i] = "_".join(parts[2:])
    elif len(parts)<3:
        cols_dict[i]=i

cols_dict2: Dict[str, str] = {
    "ID": "ID",
    "target": "target",
    "_type": "_type",
    "coinbase-premium-index_coinbase_premium_gap": "coinbase_premium_gap",
    "coinbase-premium-index_coinbase_premium_index": "coinbase_premium_index",
    "funding-rates_all_exchange_funding_rates": "funding_rates",
    "liquidations_all_exchange_all_symbol_long_liquidations": "long_liquidations",
    "liquidations_all_exchange_all_symbol_long_liquidations_usd": "long_liquidations_usd",
    "liquidations_all_exchange_all_symbol_short_liquidations": "short_liquidations",
    "liquidations_all_exchange_all_symbol_short_liquidations_usd": "short_liquidations_usd",
    "open-interest_all_exchange_all_symbol_open_interest": "open_interest",
    "taker-buy-sell-stats_all_exchange_taker_buy_ratio": "buy_ratio",
    "taker-buy-sell-stats_all_exchange_taker_buy_sell_ratio": "buy_sell_ratio",
    "taker-buy-sell-stats_all_exchange_taker_buy_volume": "buy_volume",
    "taker-buy-sell-stats_all_exchange_taker_sell_ratio": "sell_ratio",
    "taker-buy-sell-stats_all_exchange_taker_sell_volume": "sell_volume",
    "addresses-count_addresses_count_active": "active_count",
    "addresses-count_addresses_count_receiver": "receiver_count",
    "addresses-count_addresses_count_sender": "sender_count",
}
len(cols_dict)
cols_dict
df = df[cols_dict.keys()].rename(cols_dict,axis=1)

In [6]:
for i in cols_dict2.keys():
    df = df.rename(columns={i: cols_dict2[i]})


In [7]:
df.columns.nunique()

44

In [ ]:
# # Nan 값 채우기
# eda_df=df.loc[df["_type"]=="train"]
# eda_test=df.loc[df["_type"]=="test"]
# missing_values = eda_df.isnull().sum()
# missing_percentage = (missing_values / len(eda_df)) * 100
# sorted_missing_percentage = missing_percentage.sort_values(ascending=False)

# # Identify columns with missing values
# null = [col for col in eda_df.columns if missing_values[col] != 0]

# # Fill missing values with the mean of each column
# for col in null:
#     eda_df.loc[:, col] = eda_df[col].fillna(eda_df[col].mean())
    
# missing_values2 = eda_test.isnull().sum()
# missing_percentage2 = (missing_values2 / len(eda_test)) * 100
# sorted_missing_percentage2 = missing_percentage2.sort_values(ascending=False)

# null_test=[col for col in eda_test.columns if missing_values2[col] !=0 and col!='target']

# for col in null_test:
#     eda_test.loc[:,col] = eda_test[col].fillna(eda_df[col].mean())

# df=pd.concat([eda_df,eda_test],axis=0)
# df

In [8]:
df = df.assign(
    liquidation_diff=df["long_liquidations"]-df["short_liquidations"],
    liquidation_usd_diff= df['long_liquidations_usd']-df['short_liquidations_usd'],
    volume_diff=df["buy_volume"] - df["sell_volume"],
    liquidation_diffg=np.sign(df["long_liquidations"]-df["short_liquidations"]),
    liquidation_usd_diffg=np.sign(df['long_liquidations_usd']-df['short_liquidations_usd']),
    volume_diffg=np.sign(df["buy_volume"] - df["sell_volume"]),
    buy_sell_volume_ratio=df["buy_volume"] / (df["sell_volume"] + 1)
)

In [9]:
category_cols : List[str] = ["liquidation_diffg", "liquidation_usd_diffg", "volume_diffg"]
conti_cols: List[str] = [ i for i in df.columns 
                         if i not in ["ID","target","_type","liquidation_diffg", "liquidation_usd_diffg", "volume_diffg"]]
len(conti_cols)

45

In [12]:
len(df.columns)

51

In [10]:
df.columns

Index(['ID', 'target', '_type', 'block-bytes_block_bytes',
       'block-count_block_count', 'utxo-count_utxo_count',
       'difficulty_difficulty', 'active_count', 'sender_count',
       'receiver_count', 'hashrate_hashrate',
       'tokens-transferred_tokens_transferred_total',
       'tokens-transferred_tokens_transferred_mean',
       'tokens-transferred_tokens_transferred_median', 'open_interest',
       'blockreward_blockreward', 'blockreward_blockreward_usd',
       'funding_rates', 'buy_volume', 'sell_volume', 'buy_ratio', 'sell_ratio',
       'buy_sell_ratio', 'fees_fees_block_mean', 'fees_fees_block_mean_usd',
       'fees_fees_total', 'fees_fees_total_usd', 'fees_fees_reward_percent',
       'long_liquidations', 'short_liquidations', 'long_liquidations_usd',
       'short_liquidations_usd', 'coinbase_premium_gap',
       'coinbase_premium_index', 'fees-transaction_fees_transaction_mean',
       'fees-transaction_fees_transaction_mean_usd',
       'fees-transaction_fees_tran

In [11]:
def shift_feature(
    df: pd.DataFrame,
    conti_cols: List[str],
    intervals: List[int],
) -> List[pd.Series]:
    """
    연속형 변수의 shift feature 생성
    Args:
        df (pd.DataFrame)
        conti_cols (List[str]): continuous colnames
        intervals (List[int]): shifted intervals
    Return:
        List[pd.Series]
    """
    df_shift_dict = [
        df[conti_col].shift(interval).rename(f"{conti_col}_{interval}")
        for conti_col in conti_cols
        for interval in intervals
    ]
    return df_shift_dict

# 최대 24시간의 shift 피쳐를 계산
shift_list = shift_feature(
    df=df, conti_cols=conti_cols, intervals=[_ for _ in range(1, 24)]
)

In [13]:
df = pd.concat([df, pd.concat(shift_list, axis=1)], axis=1)

In [ ]:
df['open_interest']

In [ ]:
# edf=df[df['_type']=='train']
# import seaborn as sns
# sns.lineplot(x='ID',y='block-interval_block_interval',data=edf)





In [14]:
from typing import List
import pandas as pd

def rolling_feature(
   df: pd.DataFrame,
   conti_cols: List[str],
   intervals: List[int],
   funcs: List[str],
   **params,
) -> pd.DataFrame:
   """
   Create rolling features
   Args:
       df (pd.DataFrame): Sorted dataframe
       conti_cols (List[str]): continuous colnames
       intervals (List[str]): rolling window widths
       funcs (List[str]): aggregation functions e.g. ["mean", "median", "max"]
       **params: more input for rolling
   Returns:
       pd.DataFrame
   """
   df_rolling_list = [
       df[conti_col]
       .rolling(interval, **params)
       .agg({f"{conti_col}": func})
       .rename({conti_col: f"{conti_col}_{func}_{interval}"}, axis=1)
       for conti_col in conti_cols
       for interval in intervals
       for func in funcs
   ]
   return pd.concat(df_rolling_list, axis = 1)

In [15]:
conti_cols = ["coinbase_premium_gap", "coinbase_premium_index",'open_interest','block-bytes_block_bytes','funding_rates','block-count_block_count','block-interval_block_interval']
rolling_df = rolling_feature(
   df = df,
   conti_cols=conti_cols,
   intervals=[ i for i in range(2,25) if i%2==0],
   funcs=["mean", "median",'max','min'],
   min_periods = 1,
   closed = "left",
)

pd.concat([df, rolling_df], axis = 1)

,ID,target,_type,block-bytes_block_bytes,block-count_block_count,utxo-count_utxo_count,difficulty_difficulty,active_count,sender_count,receiver_count,...,block-interval_block_interval_max_20,block-interval_block_interval_min_20,block-interval_block_interval_mean_22,block-interval_block_interval_median_22,block-interval_block_interval_max_22,block-interval_block_interval_min_22,block-interval_block_interval_mean_24,block-interval_block_interval_median_24,block-interval_block_interval_max_24,block-interval_block_interval_min_24
0,2023-01-01 00:00:00,2.0,train,783554.416667,12.0,83308092.0,3.536407e+13,67987,37307,37752,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-01-01 01:00:00,1.0,train,890760.500000,4.0,83314883.0,3.536407e+13,30593,12342,20534,...,427.333333,427.333333,427.333333,427.333333,427.333333,427.333333,427.333333,427.333333,427.333333,427.333333
2,2023-01-01 02:00:00,1.0,train,558209.750000,8.0,83314090.0,3.536407e+13,33897,17737,19369,...,782.500000,427.333333,604.916667,604.916667,782.500000,427.333333,604.916667,604.916667,782.500000,427.333333
3,2023-01-01 03:00:00,1.0,train,640371.800000,5.0,83326258.0,3.536407e+13,32717,11421,23799,...,782.500000,365.125000,524.986111,427.333333,782.500000,365.125000,524.986111,427.333333,782.500000,365.125000
4,2023-01-01 04:00:00,2.0,train,712739.000000,7.0,83339168.0,3.536407e+13,45176,17320,31712,...,782.500000,365.125000,560.689583,547.566667,782.500000,365.125000,560.689583,547.566667,782.500000,365.125000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11547,2024-04-26 03:00:00,NaN,test,NaN,3.0,179820708.0,8.810419e+13,29250,18154,13601,...,2141.000000,286.571429,737.944955,516.833333,2141.000000,286.571429,748.797567,535.833333,2141.000000,286.571429
11548,2024-04-26 04:00:00,NaN,test,NaN,NaN,179833897.0,8.810419e+13,56580,31320,29096,...,2141.000000,286.571429,757.921145,535.833333,2141.000000,286.571429,757.232350,535.833333,2141.000000,286.571429
11549,2024-04-26 05:00:00,NaN,test,NaN,NaN,179851249.0,8.810419e+13,51858,34083,22094,...,2141.000000,286.571429,765.611621,627.000000,2141.000000,286.571429,743.007712,535.833333,2141.000000,286.571429
11550,2024-04-26 06:00:00,NaN,test,NaN,NaN,179852452.0,8.810419e+13,36270,26186,12668,...,2141.000000,286.571429,776.768084,658.000000,2141.000000,286.571429,750.942495,627.000000,2141.000000,286.571429


In [16]:
df=pd.concat([df, rolling_df], axis = 1)


In [17]:


# # # 타겟 변수를 제외한 변수를 forwardfill, -999로 결측치 대체
_target = df["target"]
df = df.ffill().fillna(-999).assign(target = _target)

# _type에 따라 train, test 분리
train_df = df.loc[df["_type"]=="train"].drop(columns=["_type"])
test_df = df.loc[df["_type"]=="test"].drop(columns=["_type"])

In [18]:
x_train, x_valid, y_train, y_valid = train_test_split(
    train_df.drop(["target", "ID"], axis=1), 
    train_df["target"].astype(int), 
    test_size=0.2,
    random_state=42,
    stratify=train_df["target"].astype(int)
)
x_train.shape

(7008, 1419)

In [30]:
def train_lgmClassifier(x_train,x_valid,y_train,y_valid):
    

# lgb dataset
    train_data = lgb.Dataset(x_train, label=y_train)
    valid_data = lgb.Dataset(x_valid, label=y_valid, reference=train_data)

# lgb params
    params = {
        "boosting_type": "gbdt",
        "objective": "multiclass",
        "metric": "multi_logloss",
        "num_class": 4,
        "num_leaves": 50,
        "learning_rate": 0.05,
        "n_estimators": 70,
        "random_state": 42,
        "verbose": 0,
    }

# lgb train
    lgb_model = lgb.train(
        params=params,
        train_set=train_data,
        valid_sets=valid_data,
    )

# lgb predict
    y_valid_pred = lgb_model.predict(x_valid)
    y_valid_pred_class = np.argmax(y_valid_pred, axis = 1)

# score check
    accuracy = accuracy_score(y_valid, y_valid_pred_class)
    auroc = roc_auc_score(y_valid, y_valid_pred, multi_class="ovr")

    print(f"acc: {accuracy}, auroc: {auroc}")
    return lgb_model,accuracy
lgb_model,accuracy=train_lgmClassifier(x_train,x_valid,y_train,y_valid)

/Users/sinseunghun/anaconda3/envs/my_proejct/lib/python3.9/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


acc: 0.4394977168949772, auroc: 0.6170605752985875


In [56]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, roc_auc_score
import numpy as np

def train_XGboostClassifier(x_train,x_valid,y_train,y_valid):
    
    # XGBoost params
    params = {
        "objective": "multi:softprob",  # For multiclass classification with probabilities
        "eval_metric": "mlogloss",  # Multiclass logloss
        "num_class": 4,  # Number of classes
        "learning_rate": 0.05,
        "max_depth": 6,  # XGBoost uses max_depth instead of num_leaves
        "n_estimators": 50,
        "random_state": 42,
        "verbosity": 0,
    }

    # Convert data into DMatrix (XGBoost's data structure)
    dtrain = xgb.DMatrix(x_train, label=y_train)
    dvalid = xgb.DMatrix(x_valid, label=y_valid)

# Train the model
    xgb_model = xgb.train(
        params=params,
        dtrain=dtrain,
        num_boost_round=50,
        evals=[(dvalid, "validation")],
        verbose_eval=False
    )

# XGBoost predict
    y_valid_pred = xgb_model.predict(dvalid)  # Predict probabilities for all classes
    y_valid_pred_class = np.argmax(y_valid_pred, axis=1)  # Get the class with the highest probability

# Score check
    accuracy = accuracy_score(y_valid, y_valid_pred_class)
    auroc = roc_auc_score(y_valid, y_valid_pred, multi_class="ovr")
    
    print(f"acc: {accuracy}, auroc: {auroc}")
    return xgb_model,accuracy
xgb_model,accuracy=train_XGboostClassifier(x_train,x_valid,y_train,y_valid)

acc: 0.4514840182648402, auroc: 0.6212018747617999


In [28]:
importance_dict = xgb_model.get_score(importance_type="weight")

# Convert feature importance dictionary to DataFrame
feature_importance_df = pd.DataFrame(
    list(importance_dict.items()), columns=['Feature', 'Importance']
).sort_values(by='Importance', ascending=False)

# Display the top 50 important features
print(feature_importance_df.head(50))

                                            Feature  Importance
15                                       buy_volume       134.0
16                                      sell_volume       130.0
161                               receiver_count_21        65.0
17                                        buy_ratio        65.0
889                                  volume_diff_21        63.0
109                                 active_count_15        56.0
24                               short_liquidations        55.0
152                               receiver_count_12        54.0
208  tokens-transferred_tokens_transferred_total_22        54.0
879                                  volume_diff_11        50.0
153                               receiver_count_13        48.0
392                                     buy_ratio_1        46.0
225   tokens-transferred_tokens_transferred_mean_16        45.0
391                                  sell_volume_23        45.0
524                             long_liq

In [31]:
importance = lgb_model.feature_importance()
feature_names=train_df.drop(["target", "ID"], axis = 1).columns
feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importance
})

# Sort by importance (optional)
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Display the DataFrame
print(feature_importance_df[:50])


                                             Feature  Importance
16                                       sell_volume         100
15                                        buy_volume          92
253   tokens-transferred_tokens_transferred_total_22          50
270    tokens-transferred_tokens_transferred_mean_16          47
43                                       volume_diff          44
197                                receiver_count_12          43
249   tokens-transferred_tokens_transferred_total_18          42
1055                                  volume_diff_19          40
171                                   sender_count_9          40
416                                    sell_volume_1          39
1040                                   volume_diff_4          39
17                                         buy_ratio          39
1051                                  volume_diff_15          39
8        tokens-transferred_tokens_transferred_total          38
255     tokens-transferre

# feature 걸러내기

In [32]:
# 상관계수 행렬로 변수간 상관관계 확인
# method='pearson' :  변수간 선형 상관계수 확인 
train_df = train_df.drop(columns=["ID"])
corr_mat = train_df.corr(method='pearson')
corr_mat['target'].sort_values(ascending =False)


target                   1.000000
sell_ratio               0.049467
long_liquidations        0.041983
long_liquidations_usd    0.041855
liquidation_usd_diff     0.035344
                           ...   
volume_diffg            -0.038850
buy_ratio               -0.049467
buy_sell_ratio          -0.051326
buy_sell_volume_ratio   -0.051326
volume_diff             -0.063269
Name: target, Length: 1420, dtype: float64

In [34]:
corr_threshold = 0.02

high_var_col = corr_mat[(corr_mat['target'] >= corr_threshold) | (corr_mat['target'] <= -corr_threshold )].index.to_list()
print(high_var_col)
high_var_col.remove("target")

x_train_sel = x_train[high_var_col]
x_valid_sel = x_valid[high_var_col]
x_train_sel.shape, x_valid_sel.shape

['target', 'sell_volume', 'buy_ratio', 'sell_ratio', 'buy_sell_ratio', 'long_liquidations', 'long_liquidations_usd', 'coinbase_premium_index', 'liquidation_diff', 'liquidation_usd_diff', 'volume_diff', 'liquidation_diffg', 'liquidation_usd_diffg', 'volume_diffg', 'buy_sell_volume_ratio', 'receiver_count_5', 'hashrate_hashrate_12', 'tokens-transferred_tokens_transferred_total_19', 'blockreward_blockreward_usd_12', 'buy_volume_1', 'buy_volume_19', 'sell_volume_1', 'sell_volume_19', 'fees_fees_block_mean_usd_5', 'short_liquidations_1', 'short_liquidations_2', 'short_liquidations_13', 'short_liquidations_usd_1', 'short_liquidations_usd_13', 'short_liquidations_usd_19', 'block-interval_block_interval_19', 'transactions-count_transactions_count_mean_20', 'liquidation_usd_diff_17', 'volume_diff_6', 'volume_diff_7', 'volume_diff_17', 'volume_diff_23']


((7008, 36), (1752, 36))

In [35]:
print("Lgm classifier :")
train_lgmClassifier(x_train_sel,x_valid_sel,y_train,y_valid)
print("Xgboost classifier:")
train_XGboostClassifier(x_train_sel,x_valid_sel,y_train,y_valid)

Lgm classifier :


/Users/sinseunghun/anaconda3/envs/my_proejct/lib/python3.9/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


acc: 0.4400684931506849, auroc: 0.5934593203900673
Xgboost classifier:
acc: 0.4320776255707763, auroc: 0.5927157207645082


(<xgboost.core.Booster at 0x164b1d7f0>, 0.4320776255707763)

# Down

In [36]:
from sklearn.feature_selection import VarianceThreshold

# 분산이 var_threshold보다 낮은 피쳐를 제거 
var_threshold = 1e-2
selector = VarianceThreshold(var_threshold) 
x_train_sel = selector.fit_transform(x_train)
x_valid_sel = selector.transform(x_valid)
x_train_sel.shape


(7008, 1412)

In [37]:
print("Lgm classifier :")
train_lgmClassifier(x_train_sel,x_valid_sel,y_train,y_valid)
print("Xgboost classifier:")
train_XGboostClassifier(x_train_sel,x_valid_sel,y_train,y_valid)

Lgm classifier :


/Users/sinseunghun/anaconda3/envs/my_proejct/lib/python3.9/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


acc: 0.4355022831050228, auroc: 0.6249072928012848
Xgboost classifier:
acc: 0.4389269406392694, auroc: 0.6231216815984177


(<xgboost.core.Booster at 0x177427d30>, 0.4389269406392694)

# f_classif - anova fvalue

In [57]:
from sklearn.feature_selection import f_classif, SelectKBest
feature_names = x_train.columns  
n_feature= 200
selector1 = SelectKBest(f_classif, k=n_feature)
x_train_sel = selector1.fit_transform(x_train, y_train)
x_valid_sel = selector1.transform(x_valid)

In [58]:
print("Lgm classifier :")
train_lgmClassifier(x_train_sel,x_valid_sel,y_train,y_valid)
print("Xgboost classifier:")
train_XGboostClassifier(x_train_sel,x_valid_sel,y_train,y_valid)

Lgm classifier :


/Users/sinseunghun/anaconda3/envs/my_proejct/lib/python3.9/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


acc: 0.4417808219178082, auroc: 0.6083358694833765
Xgboost classifier:
acc: 0.4434931506849315, auroc: 0.6210620238902292


(<xgboost.core.Booster at 0x28ca734f0>, 0.4434931506849315)

In [59]:
selected_features_mask = selector1.get_support()

# Get the names of the selected features
selected_features = feature_names[selected_features_mask]
selected_features[:100]

Index(['tokens-transferred_tokens_transferred_total',
       'tokens-transferred_tokens_transferred_mean', 'open_interest',
       'buy_volume', 'sell_volume', 'buy_ratio', 'sell_ratio',
       'buy_sell_ratio', 'long_liquidations', 'short_liquidations',
       'long_liquidations_usd', 'short_liquidations_usd',
       'coinbase_premium_gap', 'coinbase_premium_index', 'volume_diff',
       'liquidation_diffg', 'liquidation_usd_diffg', 'volume_diffg',
       'buy_sell_volume_ratio',
       'tokens-transferred_tokens_transferred_total_1',
       'tokens-transferred_tokens_transferred_total_2',
       'tokens-transferred_tokens_transferred_total_3',
       'tokens-transferred_tokens_transferred_total_4',
       'tokens-transferred_tokens_transferred_total_8',
       'tokens-transferred_tokens_transferred_total_22', 'open_interest_1',
       'open_interest_2', 'open_interest_3', 'open_interest_4',
       'open_interest_5', 'open_interest_6', 'open_interest_7',
       'open_interest_8', 'ope

## SelectFromModel ExtraTreesClassifier

In [62]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import ExtraTreesClassifier

# 선택할 피쳐의 갯수
num_max_features = 200

# 50개의 결정 트리를 갖는 ExtraTreesClassifier 모델을 생성
model_sel = ExtraTreesClassifier(n_estimators=50).fit(x_train, y_train)

# 학습된 ExtraTreesClassifier 모델(model_sel)을 사용하여 feature selection을 수행
selector = SelectFromModel(model_sel, prefit=True, max_features=num_max_features)
x_train_sel = selector.transform(x_train)
x_valid_sel = selector.transform(x_valid)

/Users/sinseunghun/anaconda3/envs/my_proejct/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/sinseunghun/anaconda3/envs/my_proejct/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


In [63]:
print("Lgm classifier :")
train_lgmClassifier(x_train_sel,x_valid_sel,y_train,y_valid)
print("Xgboost classifier:")
train_XGboostClassifier(x_train_sel,x_valid_sel,y_train,y_valid)

/Users/sinseunghun/anaconda3/envs/my_proejct/lib/python3.9/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Lgm classifier :
acc: 0.4275114155251142, auroc: 0.5904757814679427
Xgboost classifier:
acc: 0.4457762557077626, auroc: 0.5858536678679428


(<xgboost.core.Booster at 0x28ca73370>, 0.4457762557077626)

# Cross valiadtion


In [64]:
RANDOM_SEED=42

In [65]:
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import SelectKBest, f_classif

# Initialize StratifiedKFold to maintain class proportions
k_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# List to store predictions
pred_lgm = []
pred_xgb = []

# Perform cross-validation with stratification
for train_idx, val_idx in k_fold.split(train_df.drop(["target"], axis=1), train_df["target"]):
    # Train and validation sets
    x_tr = train_df.drop(["target"], axis=1).iloc[train_idx]
    y_tr = train_df['target'].iloc[train_idx]
    x_val = train_df.drop(["target"], axis=1).iloc[val_idx]
    y_val = train_df['target'].iloc[val_idx]

    # Select top 200 features using SelectKBest with f_classif
    n_feature = 200
    selector1 = SelectKBest(f_classif, k=n_feature)
    x_tr_sel = selector1.fit_transform(x_tr, y_tr)
    x_val_sel = selector1.transform(x_val)
    _,accuracy=train_lgmClassifier(x_tr_sel, x_val_sel, y_tr, y_val)
    # Train and get predictions for LGBM and XGBoost
    pred_lgm.append(accuracy)
    _,accuracy=train_XGboostClassifier(x_tr_sel, x_val_sel, y_tr, y_val)
    pred_xgb.append(accuracy)


/Users/sinseunghun/anaconda3/envs/my_proejct/lib/python3.9/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


acc: 0.4400684931506849, auroc: 0.6150620275827994
acc: 0.4412100456621005, auroc: 0.6299835505981054


/Users/sinseunghun/anaconda3/envs/my_proejct/lib/python3.9/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


acc: 0.4269406392694064, auroc: 0.6287895461440638
acc: 0.4452054794520548, auroc: 0.6338856323333695


/Users/sinseunghun/anaconda3/envs/my_proejct/lib/python3.9/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


acc: 0.4263698630136986, auroc: 0.6122184134619
acc: 0.4275114155251142, auroc: 0.6161333354723829


/Users/sinseunghun/anaconda3/envs/my_proejct/lib/python3.9/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


acc: 0.4343607305936073, auroc: 0.6187462076452501
acc: 0.45662100456621, auroc: 0.6132083987533256


/Users/sinseunghun/anaconda3/envs/my_proejct/lib/python3.9/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


acc: 0.4172374429223744, auroc: 0.6225679546115073
acc: 0.4337899543378995, auroc: 0.6303566747654855


In [67]:
print(np.mean(pred_lgm))
print(np.mean(pred_xgb))

0.4289954337899543
0.4408675799086758


In [ ]:
# x_test=test_df.drop(columns=['target','ID'])
# x_test=selector1.transform(x_test)

In [85]:
# performance 체크후 전체 학습 데이터로 다시 재학습
# x_train = train_df.drop(["target"], axis = 1)
# y_train = train_df["target"].astype(int)
# n_feature= 200
# selector1 = SelectKBest(f_classif, k=n_feature)
# x_train_sel = selector1.fit_transform(x_train, y_train)
# params = {
#         "objective": "multi:softprob",  # For multiclass classification with probabilities
#         "eval_metric": "mlogloss",  # Multiclass logloss
#         "num_class": 4,  # Number of classes
#         "learning_rate": 0.05,
#         "max_depth": 6,  # XGBoost uses max_depth instead of num_leaves
#         "random_state": 42,
#         "verbosity": 0,
#     }
# dtrain = xgb.DMatrix(x_train_sel, label=y_train)

# # Step 5: Train the model using xgb.train
# xgb_model = xgb.train(
#     params=params,
#     dtrain=dtrain,
#     num_boost_round=50,
#     verbose_eval=False  # Set to True if you want to see progress
# )

# # Step 6: Prepare the test set by dropping unnecessary columns
# x_test = test_df.drop(columns=['target', 'ID'])

# # Step 7: Apply the same feature selection to the test set
# x_test_sel = selector1.transform(x_test)

# # Step 8: Convert the test set to DMatrix (required for prediction)
# dtest = xgb.DMatrix(x_test_sel)

# # Step 9: Predict class probabilities for the test set
# y_test_pred = xgb_model.predict(dtest)

# # Step 10: Get the predicted class by selecting the class with the highest probability
# y_test_pred_class = np.argmax(y_test_pred, axis=1)



In [86]:
x_train = train_df.drop(["target", "ID"], axis = 1)
y_train = train_df["target"].astype(int)
train_data = lgb.Dataset(x_train, label=y_train)
lgb_model = lgb.train(
    params=params,
    train_set=train_data,
)

array([[4.04390475e+04, 1.03455406e+10, 2.32407067e+08, ...,
        2.18950000e+03, 6.78528290e+02, 2.18950000e+03],
       [1.89696525e+04, 1.04016515e+10, 2.50681096e+08, ...,
        2.18950000e+03, 7.04315790e+02, 2.18950000e+03],
       [1.20699077e+04, 1.03959417e+10, 1.07727948e+08, ...,
        2.18950000e+03, 7.00642811e+02, 2.18950000e+03],
       ...,
       [2.22622436e+04, 1.48683608e+10, 3.20161937e+08, ...,
        2.14100000e+03, 7.43007712e+02, 2.14100000e+03],
       [3.33492974e+04, 1.48683608e+10, 3.20161937e+08, ...,
        2.14100000e+03, 7.50942495e+02, 2.14100000e+03],
       [5.73043530e+04, 1.48683608e+10, 3.20161937e+08, ...,
        2.14100000e+03, 7.49428002e+02, 2.14100000e+03]])

In [ ]:
y_test_pred = lgb_model.predict(test_df.drop(["target", "ID"], axis = 1))
y_test_pred_class = np.argmax(y_test_pred, axis = 1)

In [87]:
# output file 할당후 save 
submission_df = submission_df.assign(target = y_test_pred_class)
submission_df.to_csv("output.csv", index=False)